In [1]:
import pandas as pd
import numpy as np

## Extracting movies feature from 2018 movies form Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [3]:
df = df1.append(df2.append(df3.append(df4, ignore_index=True), ignore_index=True), ignore_index=True)

In [4]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2]
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3]
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4]
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5]
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6]
...,...,...,...,...,...,...
265,DECEMBER,25,Holmes & Watson,Columbia Pictures / Gary Sanchez Productions,Etan Cohen (director/screenplay); Will Ferrell...,[163]
266,DECEMBER,25,Vice,Annapurna Pictures / Plan B Entertainment,Adam McKay (director/screenplay); Christian Ba...,[137]
267,DECEMBER,25,On the Basis of Sex,Focus Features,Mimi Leder (director); Daniel Stiepleman (scre...,[226]
268,DECEMBER,25,Destroyer,Annapurna Pictures,"Karyn Kusama (director); Phil Hay, Matt Manfre...",[258]


### Deling with Null values

In [5]:
df.isnull().sum()

Opening               0
Opening.1             0
Title                 0
Production company    0
Cast and crew         0
Ref.                  1
dtype: int64

In [6]:
!pip install tmdbv3api

In [7]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'e3dce8cf7d87efd38c89a43d9fe6d597'

In [8]:
from tmdbv3api import Movie
tmdb_movie = Movie()

def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, tmdb.api_key))
    json_data = response.json()
    if json_data['genres']:
        str_ = " "
        for i in range(0, len(json_data['genres'])):
            genres.append(json_data['genres'][i]['name'])
        return str_.join(genres)
    else:
        np.NaN

In [9]:
df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))

In [10]:
df.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,genres
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],Horror Mystery Thriller
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],Thriller Drama
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4],Action Thriller
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5],Drama History Western
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6],Action Thriller


In [11]:
df_2018 = df[['Title','Cast and crew','genres']]

In [12]:
df_2018.head()

,Title,Cast and crew,genres
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller


In [13]:
df_2018['Cast and crew'][0].partition('screenplay);')[-1].split(', ')[0].strip()
# print(partitioned_string)

'Lin Shaye'

In [14]:
def get_director(x):
    if " (director)" in x:
        return x.partition(' (director)')[0]
    elif " (directors)" in x:
        return x.partition(' (directors)')[0]
    else:
        return x.partition(' (director/screenplay)')[0]

In [15]:
df_2018['director_name'] = df_2018['Cast and crew'].map(lambda x: get_director(x))

<ipython-input-15-4838efec492a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['director_name'] = df_2018['Cast and crew'].map(lambda x: get_director(x))


In [16]:
def get_actor1(x):
    return ((x.partition("screenplay); ")[-1]).split(", ")[0]).strip()

In [17]:
df_2018['actor1'] = df_2018['Cast and crew'].map(lambda x: get_actor1(x))

<ipython-input-17-4c5abb571cf8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['actor1'] = df_2018['Cast and crew'].map(lambda x: get_actor1(x))


In [18]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [19]:
df_2018['actor2'] = df_2018['Cast and crew'].map(lambda x: get_actor2(x))

<ipython-input-19-050fb08fc984>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2018['actor2'] = df_2018['Cast and crew'].map(lambda x: get_actor2(x))


In [20]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [21]:
df_2018['actor3'] = df_2018['Cast and crew'].map(lambda x: get_actor3(x))

In [22]:
df_2018.drop(columns={'Cast and crew'}, inplace=True)

In [23]:
df_2018.head()

,Title,genres,director_name,actor1,actor2,actor3
0,Insidious: The Last Key,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,The Strange Ones,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,Stratton,Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan
3,Sweet Country,Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,NaN
4,The Commuter,Action Thriller,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson


### Deling with NaN values

In [24]:
df_2018.isna().sum()

Title             0
genres            3
director_name     0
actor1            0
actor2            9
actor3           26
dtype: int64

In [25]:
df_2018.fillna('', inplace=True)

In [26]:
df_2018.isna().sum()

Title            0
genres           0
director_name    0
actor1           0
actor2           0
actor3           0
dtype: int64

In [27]:
df_2018 = df_2018.rename(columns={'Title':'title', 'director_name':'director'})

In [28]:
df_2018['comb'] = df_2018.actor1 + ' ' + df_2018.actor2 + ' ' + df_2018.actor3 + ' ' + df_2018.director + ' ' + df_2018.genres

In [29]:
df_2018['title'] = df_2018['title'].str.lower()

In [30]:
df_2018

,title,genres,director,actor1,actor2,actor3,comb
0,insidious: the last key,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
1,the strange ones,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Alex Pettyfer James Freedson-Jackson Emily Alt...
2,stratton,Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Dominic Cooper Austin Stowell Gemma Chan Simon...
3,sweet country,Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,,Bryan Brown Sam Neill Warwick Thornton Drama ...
4,the commuter,Action Thriller,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...
...,...,...,...,...,...,...,...
265,holmes & watson,Mystery Adventure Comedy Crime,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall,Will Ferrell John C. Reilly Rebecca Hall Etan ...
266,vice,Thriller Science Fiction Action Adventure,Adam McKay,Christian Bale,Amy Adams,Steve Carell,Christian Bale Amy Adams Steve Carell Adam McK...
267,on the basis of sex,Drama History,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux,Felicity Jones Armie Hammer Justin Theroux Mim...
268,destroyer,Thriller Crime Drama Action,Karyn Kusama,Nicole Kidman,Sebastian Stan,Toby Kebbell,Nicole Kidman Sebastian Stan Toby Kebbell Kary...


In [31]:
df_2018.to_csv("wiki_2018.csv", index=False)